In [2]:
import pandas as pd
import numpy as np

In [40]:
movie = pd.read_csv("..\data\processed\movies_clean.csv")
movies = movie.copy()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\yasha\AppData\Local\Temp\ipykernel_23404\2665026983.py:1: SyntaxWarning: invalid escape sequence '\d'
  movie = pd.read_csv("..\data\processed\movies_clean.csv")
C:\Users\yasha\AppData\Local\Temp\ipykernel_23404\2665026983.py:1: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  movie = pd.read_csv("..\data\processed\movies_clean.csv")


In [41]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318158 entries, 0 to 318157
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    318158 non-null  int64  
 1   title                 318158 non-null  object 
 2   vote_average          318158 non-null  float64
 3   vote_count            318158 non-null  int64  
 4   release_date          306070 non-null  object 
 5   runtime               318158 non-null  int64  
 6   backdrop_path         318158 non-null  object 
 7   imdb_id               165872 non-null  object 
 8   popularity            318158 non-null  float64
 9   poster_path           318158 non-null  object 
 10  genres                260695 non-null  object 
 11  production_countries  244043 non-null  object 
 12  trailer_link          76371 non-null   object 
 13  combined_text         318158 non-null  object 
dtypes: float64(2), int64(3), object(9)
memory usage: 34.

In [42]:
# Convert all object/string columns to lowercase
for col in movies.select_dtypes(include='object').columns:
    movies[col] = movies[col].str.lower()


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
vector = TfidfVectorizer(stop_words='english', max_features=20000)
model = vector.fit_transform(movie['combined_text'])

In [45]:
from sklearn.decomposition import TruncatedSVD

In [46]:
svd = TruncatedSVD(n_components=200, random_state=42)
vector_reduced = svd.fit_transform(model)

In [47]:
import pickle

In [ ]:
pickle.dump(vector, open("models/vector.pkl", "wb"))
pickle.dump(vector_reduced, open("models/vector_reduced.pkl", "wb"))
pickle.dump(svd, open("models/svd_model.pkl", "wb"))


In [49]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies(query, top_n=10):

    movies_filtered = movie[(movie["vote_average"] > 2) & (movie["vote_count"] > 40)]
    query_vec = vector.transform([query.lower()])
    query_vec_reduced = svd.transform(query_vec)
    similarity = cosine_similarity(query_vec_reduced, vector_reduced[movies_filtered.index]).flatten()
    top_indices = similarity.argsort()[::-1][2:top_n+1]
    recommended = movies_filtered.iloc[top_indices][["id",'imdb_id','title','vote_average','vote_count','genres','backdrop_path','poster_path','trailer_link']].copy()
    recommended['similarity'] = similarity[top_indices]
    
    return recommended

##dilwale dulhaniya le jayenge
query_result = recommend_movies("Comedy  Drama india")
query_result.head(10)


,id,imdb_id,title,vote_average,vote_count,genres,backdrop_path,poster_path,trailer_link,similarity
16067,439128,tt6484982,Newton,7.074,121,"Comedy, Drama",https://image.tmdb.org/t/p/original/5aNFEcMFjD...,https://image.tmdb.org/t/p/original/ubzHA1aXUs...,https://www.youtube.com/watch?v=yU6zMPFd4UU,0.658063
11793,135718,tt2283748,OMG: Oh My God!,7.200,199,"Comedy, Drama, Fantasy",https://image.tmdb.org/t/p/original/kMUsTD4Dco...,https://image.tmdb.org/t/p/original/wUnwafB1Zv...,https://www.youtube.com/watch?v=eSfJ9NTE0OE,0.652570
22370,65521,tt1926313,Pyaar Ka Punchnama,6.900,71,"Drama, Comedy, Romance",https://image.tmdb.org/t/p/original/yxuhI3YYk3...,https://image.tmdb.org/t/p/original/vKCuXtM3hp...,https://www.youtube.com/watch?v=DkWWklnZ-Tc,0.651847
30878,320873,tt4088588,Dhanak,7.300,41,"Drama, Comedy",https://image.tmdb.org/t/p/original/8Q09GgC37I...,https://image.tmdb.org/t/p/original/cfn8cy2VGK...,https://www.youtube.com/watch?v=7uSpaly_8JQ,0.643975
25614,74458,tt1740710,Mere Brother Ki Dulhan,6.202,57,"Drama, Comedy, Romance",https://image.tmdb.org/t/p/original/uVmMIhggCA...,https://image.tmdb.org/t/p/original/hz45DxF4CV...,NaN,0.637836
29541,479918,tt7399470,Qarib Qarib Singlle,7.000,44,"Comedy, Drama, Romance",https://image.tmdb.org/t/p/original/jhkeinQ03W...,https://image.tmdb.org/t/p/original/yKEKZ0uTpK...,https://www.youtube.com/watch?v=MU4QL-VwDlc,0.637785
27781,25499,tt0085743,Jaane Bhi Do Yaaro,6.800,49,Comedy,https://image.tmdb.org/t/p/original/1BbgYWI4Dz...,https://image.tmdb.org/t/p/original/m4K4Nj3Axr...,NaN,0.631570
26796,614179,tt10280276,Coolie No. 1,5.596,52,"Comedy, Drama",https://image.tmdb.org/t/p/original/bNixlnIPYJ...,https://image.tmdb.org/t/p/original/ahkaPWW2SU...,NaN,0.617479
20445,117751,tt2218988,Ustad Hotel,7.561,82,"Drama, Comedy",https://image.tmdb.org/t/p/original/87pmx2Unwv...,https://image.tmdb.org/t/p/original/88QYspKRAr...,https://www.youtube.com/watch?v=Z1m3y4L1yzY,0.616889
